In [1]:
import csv

def print_board(board):
    """Affiche le plateau de jeu."""
    for row in board:
        print(" | ".join(row))
        print("-" * 5)

def check_winner(board):
    """Vérifie s'il y a un gagnant."""
    # Vérification des lignes et des colonnes
    for i in range(3):
        if board[i][0] == board[i][1] == board[i][2] and board[i][0] != ' ':
            return board[i][0]
        if board[0][i] == board[1][i] == board[2][i] and board[0][i] != ' ':
            return board[0][i]
    # Vérification des diagonales
    if board[0][0] == board[1][1] == board[2][2] and board[0][0] != ' ':
        return board[0][0]
    if board[0][2] == board[1][1] == board[2][0] and board[0][2] != ' ':
        return board[0][2]
    return None

def is_board_full(board):
    """Vérifie si le plateau est plein."""
    for row in board:
        for cell in row:
            if cell == ' ':
                return False
    return True

def get_possible_moves(board):
    """Retourne la liste des coups possibles."""
    moves = []
    for i in range(3):
        for j in range(3):
            if board[i][j] == ' ':
                moves.append((i, j))
    return moves

def minimax(board, depth, is_maximizing, alpha, beta):
    """Algorithme Minimax avec élagage alpha-bêta."""
    winner = check_winner(board)
    if winner == 'X':
        return 1
    if winner == 'O':
        return -1
    if is_board_full(board):
        return 0

    if is_maximizing:
        best_score = -float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'X'
            score = minimax(board, depth + 1, False, alpha, beta)
            board[move[0]][move[1]] = ' '
            best_score = max(score, best_score)
            alpha = max(alpha, best_score)
            if beta <= alpha:
                break
        return best_score
    else:
        best_score = float('inf')
        for move in get_possible_moves(board):
            board[move[0]][move[1]] = 'O'
            score = minimax(board, depth + 1, True, alpha, beta)
            board[move[0]][move[1]] = ' '
            best_score = min(score, best_score)
            beta = min(beta, best_score)
            if beta <= alpha:
                break
        return best_score

def find_best_move(board, player):
    """Trouve le meilleur coup pour un joueur donné."""
    best_score = -float('inf') if player == 'X' else float('inf')
    best_move = None
    for move in get_possible_moves(board):
        board[move[0]][move[1]] = player
        if player == 'X':
            score = minimax(board, 0, False, -float('inf'), float('inf'))
            if score > best_score:
                best_score = score
                best_move = move
        else:
            score = minimax(board, 0, True, -float('inf'), float('inf'))
            if score < best_score:
                best_score = score
                best_move = move
        board[move[0]][move[1]] = ' '
    return best_move

dataset = []
visited_states = set()

def generate_dataset(board, player):
    """Génère récursivement le dataset."""
    board_str = "".join(["".join(row) for row in board])
    if board_str in visited_states or check_winner(board) or is_board_full(board):
        return

    visited_states.add(board_str)
    best_move = find_best_move(board, player)

    # Ajout de l'état et du meilleur coup au dataset
    if best_move:
        dataset.append([board_str, f"{best_move[0]},{best_move[1]}"])

    # Exploration des prochains états
    for move in get_possible_moves(board):
        board[move[0]][move[1]] = player
        generate_dataset(board, 'O' if player == 'X' else 'X')
        board[move[0]][move[1]] = ' '

# Initialisation et lancement de la génération
initial_board = [[' ' for _ in range(3)] for _ in range(3)]
generate_dataset(initial_board, 'X')

# Exportation en CSV
with open('tictactoe_dataset.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['board_state', 'best_move'])
    writer.writerows(dataset)

print(f"Dataset généré avec {len(dataset)} états et sauvegardé dans 'tictactoe_dataset.csv'")

Dataset généré avec 4520 états et sauvegardé dans 'tictactoe_dataset.csv'
